# Machine Learning Automation with Proactive Jupyter Kernel
The ActiveEon Jupyter Kernel adds a kernel backend to Jupyter.

This kernel interfaces directly with the ProActive scheduler and constructs tasks and workflows to execute them on the fly.

With this interface, users can run their code locally and test it using a native python kernel, and by a simple switch to ProActive kernel, run it on remote public or private infrastructures without having to modify the code.

See https://github.com/ow2-proactive/proactive-jupyter-kernel for more information.

As a quick start, we recommend the user to run the `#%help()` pragma using the following script:

In [ ]:
#%help()

## Connection

If you are trying ProActive for the first time, sign up on the [try platform](https://try.activeeon.com/signup.html).

Once you receive your login and password, connect to the trial platform using the `#%connect()` pragma.

For more information, type: `#%help(pragma=connect)`

In [ ]:
#%connect(url=https://try.activeeon.com:8443)

## Runtime environment definition

The `#%runtime_env()` pragma enable user to define the runtime environment for pipeline execution.

The user can select the container type (docker, podman, singularity), the container image, and mount local directories inside container.

For more information, type: `#%help(pragma=runtime_env)`

In [ ]:
#%runtime_env(type=docker,image=activeeon/dlm3,mount_host_path=/shared,mount_container_path=/shared,debug=false,verbose=false,force=off)

## Importing libraries
The main difference between the ProActive and 'native language' kernels resides in the way the memory is accessed
during blocks execution. In a common native language kernel, the whole script code (all the notebook blocks) is
locally executed in the same shared memory space; whereas the ProActive kernel will execute each created task in an
independent process. In order to facilitate the transition from native language to ProActive kernels, we included the
pragma `#%import()`. This pragma gives the user the ability to add libraries that are common to all created tasks, and
thus relative distributed processes, that are implemented in the same native script language.

The import pragma is used as follows:

`#%import([language=SCRIPT_LANGUAGE])`.

Example:

```python
#%import(language=Python)
import os
import pandas
```

NOTE: If the language is not specified, Python is considered as default language.

In [ ]:
#%import()
import pandas as pd
import pickle
import bz2

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Creating tasks

### Creating the _import_ task

In [ ]:
#%task(name=import,export=[dataframe_json])
boston = load_boston()
dataframe_load = pd.DataFrame(boston.data)
dataframe_load.columns = boston.feature_names 
data_label = boston.target
dataframe = dataframe_load.assign(LABEL=data_label)

dataframe_json = dataframe.to_json(orient='split').encode()
compressed_data = bz2.compress(dataframe_json)
dataframe.head()

### Creating the _split_ task 

In [ ]:
#%task(name=split,dep=[import],import=[dataframe_json],export=[X_train_json,X_test_json])
dataframe = pd.read_json(dataframe_json, orient='split')

X_train, X_test = train_test_split(dataframe, test_size=30)

X_train_json = X_train.to_json(orient='split').encode()
X_test_json = X_test.to_json(orient='split').encode()

### Creating the _train_ task

In [ ]:
#%task(name=train,dep=[split],import=[X_train_json],export=[filename,columns])
# Fit the model on 33%
columns = "LABEL"

X_train = pd.read_json(X_train_json, orient='split')

dataframe_train = X_train.drop(columns, axis=1, inplace=False)
dataframe_label = X_train.filter(columns, axis=1)

model = LinearRegression()
model.fit(dataframe_train, dataframe_label)

# save the model to disk
filename = '/shared/finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

### Creating the _predict_ task 

In [ ]:
#%task(name=predict,dep=[train,split],import=[columns,filename,X_test_json])
X_test = pd.read_json(X_test_json, orient='split')

dataframe_test = X_test.drop(columns, axis=1, inplace=False)
dataframe_label = X_test.filter(columns, axis=1)
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(dataframe_test, dataframe_label)

print('The prediction result is = ' + str(result))

### Visualizing the job pipeline

In [ ]:
#%draw_job()

### Submitting the job to the scheduler

To submit the job to the ProActive Scheduler, the user has to use the `#%submit_job()` pragma:

```python
#%submit_job()
```

If the job is not created, or is not up-to-date, the `#%submit_job()` creates a new job named as the old one.
To provide a new name, use the same pragma and provide a name as parameter:

```python
#%submit_job([name=JOB_NAME])
```

If the job's name is not set, the ProActive kernel uses the current notebook name, if possible, or gives a random one.

In [ ]:
#%submit_job(name=ML_Pipeline_Example)

### Getting results and outputs

After the execution of a ProActive workflow, two outputs can be obtained,
* results: values that have been saved in the 
[task result variable](https://doc.activeeon.com/latest/user/ProActiveUserGuide.html#_task_result),
* console outputs: classic outputs that have been displayed/printed 

To get task results, please use the `#%get_task_result()` pragma by providing the task name, and either the job ID or
the job name:

```python
#%get_task_result([job_id=JOB_ID], [job_name=JOB_NAME], task_name=TASK_NAME)
```

The result(s) of all the tasks of a job can be obtained with the `#%get_job_result()` pragma, by providing the job name
or the job ID:

```python
#%get_job_result([job_id=JOB_ID], [job_name=JOB_NAME])
```

To get and display console outputs of a task, you can use the `#%print_task_output()` pragma in the following
way:

```python
#%print_task_output([job_id=JOB_ID], [job_name=JOB_NAME], task_name=TASK_NAME)
```

Finally, the  `#%print_job_output()` pragma allows to print all job outputs, by providing the job name or the job ID:

```python
#%print_job_output([job_id=JOB_ID], [job_name=JOB_NAME])
```

NOTE: If neither `job_name` nor the `job_id` are provided, the last submitted job is selected by default. 

In [ ]:
#%print_task_output(task_name=predict)